In [6]:
import pandas as pd
import numpy as np
import math

In [7]:
# Creation of the dataframe and Dropping the useless features
to_open = "Corr-20240316_TheBlaze.csv"
output_name = to_open.split('-')[1]
date = output_name.split('_')[0][:4] + "-" + output_name.split('_')[0][4:6] + "-" + output_name.split('_')[0][6:8]
print(date)
df = pd.read_csv(f"weekly_corrected//{to_open}")
df

2024-03-16


,Unnamed: 0,time,number,t2m
0,0,2024-03-16 00:00:00,1,284.158582
1,1,2024-03-16 00:00:00,2,284.101156
2,2,2024-03-16 00:00:00,3,284.069274
3,3,2024-03-16 00:00:00,4,284.199158
4,4,2024-03-16 00:00:00,5,284.273234
...,...,...,...,...
1445,1445,2024-03-23 00:00:00,46,283.045167
1446,1446,2024-03-23 00:00:00,47,279.778507
1447,1447,2024-03-23 00:00:00,48,282.497487
1448,1448,2024-03-23 00:00:00,49,280.191393


In [8]:
df=df[['time','number','t2m']]

In [9]:
df

,time,number,t2m
0,2024-03-16 00:00:00,1,284.158582
1,2024-03-16 00:00:00,2,284.101156
2,2024-03-16 00:00:00,3,284.069274
3,2024-03-16 00:00:00,4,284.199158
4,2024-03-16 00:00:00,5,284.273234
...,...,...,...
1445,2024-03-23 00:00:00,46,283.045167
1446,2024-03-23 00:00:00,47,279.778507
1447,2024-03-23 00:00:00,48,282.497487
1448,2024-03-23 00:00:00,49,280.191393


In [10]:
df['t2m']=df['t2m']-273.15

/var/folders/lw/bnynzkxj0cq2wnbmqw0rvyzh0000gn/T/ipykernel_90639/836502028.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['t2m']=df['t2m']-273.15


In [11]:
# Creation of the results Dataframes which will be made into one df later on
result_t2m  = pd.DataFrame(columns=['forecast_date','target','horizon','q0.025','q0.25','q0.5','q0.75','q0.975'])

In [12]:
times=df.time.unique()
number_of_hours=0
#defined earlier
#date = "2023-03-15"
row_index=0
for t in times:
  df_=df[df['time']==t]
  t2m = []
  t2m=df_.t2m.values


  horizon = str(number_of_hours)+' hour'
  result_t2m.loc[row_index]  = [date, 't2m', horizon , np.quantile(t2m,0.025),np.quantile(t2m,0.25),np.quantile(t2m,0.5),np.quantile(t2m,0.75),np.quantile(t2m,0.975)]
  row_index=row_index+1
  number_of_hours = number_of_hours + 6


In [13]:
# Dataframes concatination and result exportation
result_t2m = pd.DataFrame.to_csv(result_t2m,f"weekly_corrected/{output_name}", index=False)

In [14]:
result_t2m